# Residential demand estimation

In [ ]:
%matplotlib inline

In [ ]:
# import packages

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../data/raw/Residential/AUS_data.csv",
                index_col=['Economy','Year'])

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
# split in to training and test sets

feature_list = ["GDP","CSGDP","POP","POPU","GDPC"]

train = (df.loc['01_AUS',:]
           .loc[1980:2000])
train.head()

In [ ]:
test = (df.loc['01_AUS',:]
          .loc[2001:2016])
test.tail()

In [ ]:
# training
y = train.loc[:,"RETT"]
X_train = train.loc[:,feature_list]

# test
X_test = test.loc[:,feature_list]

In [ ]:
X_train.head()

In [ ]:
X_test.tail()

In [ ]:
model = sm.OLS(y,X_train)

In [ ]:
fit_model = model.fit()

In [ ]:
fit_model.summary()

In [ ]:
# test model

results = fit_model.predict(X_test)

In [ ]:
results

In [ ]:
fig, ax = plt.subplots()
ax.plot(train.loc[:,'RETT'], 'o', label="Train")
ax.plot(results, 'o', label="Test")
ax.plot(df.loc['01_AUS',"RETT"], '-g', label="Actual")
ax.legend(loc="best")

#### next steps

- add lags?

## ARIMA

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df = pd.read_csv("../data/raw/Residential/AUS_simple.csv",
                index_col=['Economy','Date'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
feature_list = ["GDP","CSGDP","POP","POPU","GDPC"]

train = (df.loc['01_AUS',:]
           .iloc[1:21]
           .drop(columns="Year"))
train.tail()

In [ ]:
y = train[['RETT']]
X_train = train.drop(columns=y)
X_train.tail()

In [ ]:
model = ARIMA(y, (0,1,0),X_train, freq='A')

In [ ]:
model_fit = model.fit(disp=0)

In [ ]:
print(model_fit.summary())

In [ ]:
test = (df.loc['01_AUS',:]
           .iloc[21:37]
           .drop(columns="Year"))
test.tail()

In [ ]:
X_test = test.drop(columns="RETT")
X_test.head()

In [ ]:
forecast = model_fit.forecast(steps=16,exog=X_test)

In [ ]:
forecast = model_fit.forecast?

In [ ]:
forecast = model_fit.forecast

In [ ]:
forecast = model_fit.predict

In [ ]:
ARIMA?

# Prophet

In [ ]:
from fbprophet import Prophet

In [ ]:
df = pd.read_csv("../data/raw/Residential/AUS_data.csv")

In [ ]:
df.head()

In [ ]:
df['ds'] = pd.to_datetime(df.Year,format='%Y')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
simple_df = df[['RETT','GDP','POP','ds']]

In [ ]:
simple_df.head()

In [ ]:
simple_df = simple_df.rename(columns={"RETT":"y"})
simple_df.head()

In [ ]:
m = Prophet(daily_seasonality=False,
            weekly_seasonality=False,
            yearly_seasonality=False,
            seasonality_mode='additive',
            growth='linear')

m.add_regressor('GDP')
m.add_regressor('POP')
m.fit(simple_df)

In [ ]:
# add future GDP

df_future_macro = pd.read_csv('../data/raw/Industry/MacroAssumptions_datetime.csv',
                             index_col=['Economy'])

df_future_macro = df_future_macro.loc['AUS',:]
df_future_macro = df_future_macro.reset_index(drop=True)

df_future_macro.head()

In [ ]:
df_future_macro = df_future_macro.rename(columns={"Population":"POP"})
df_future_macro.head()

In [ ]:
future = m.make_future_dataframe(periods=34,
                                 freq = 'Y',
                                 include_history=True)
future.head()

In [ ]:
future.tail()

In [ ]:
df1 = simple_df.drop(columns=['y','ds'])
df1.head()

In [ ]:
df2 = df_future_macro.drop(columns=['Year'])
df2.head()

In [ ]:
df_future = pd.concat([df1,df2],
                      ignore_index=True)

df_future.head()

In [ ]:
df_future.tail()

In [ ]:
future = future.join(df_future)

In [ ]:
future.head()

In [ ]:
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
future.iloc[20:40,:]

# SciKit-Learn

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(simple_df[["GDP","POP"]],simple_df[['ds']])

In [ ]:
model.fit(X_train,y)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
model2 = LinearRegression()

In [ ]:
model2.fit(simple_df[["GDP","POP"]],simple_df[['y']])

In [ ]:
predictions2 = model2.predict(future[["GDP","POP"]])
predictions2

In [ ]:
fig, ax = plt.subplots()
ax.plot(future[['ds']],predictions2, 'o', label="Predictions")
ax.plot(simple_df[['ds']],simple_df[["y"]], 'o', label="Actual")
ax.legend(loc="best")

# Using own functions

In [ ]:
def run_regression(models, economies, df, x, y):
    """
    Perform linear regression for one or multiple economies.
    economy = list of economies
    models = {economy: LinearRegression() for economy in economies}
    The function returns a dictionary of economy-model pairs. That is,
    each economy will have its own set of coefficients.
    """
    for economy, model in models.items():
            (model.fit(df.loc[economy, x],
                df.loc[economy, y]))
    return models  

In [ ]:
def run_prediction(models, economies, df, ResultsColumn):
    """
    Use coefficients from run_regression to generate predictions.
    Pass a dataframe df with the X and Y data. 
    ResultsColumn = name of prediction results
    """
    df_list =[]
    # run predictions
    for economy, model in models.items():
            #years = df['Year']
            #years.reset_index(drop=True, inplace=True)
            prediction = model.predict(df.loc[economy,:])
            df_name = pd.DataFrame(prediction, columns=ResultsColumn)
            #df_name.insert(loc=0,column='Year',value=years)
            df_name.insert(loc=0,column='Economy',value=economy)
            #df_list.append(df_name)
            
            # adding to the input df
            df2 = df.loc[economy,:]
            df2.insert(loc=0,column="Results",value=prediction)
            df2.insert(loc=0,column='Economy',value=economy)
            df2 = df2.reset_index()
            df2 = df2.set_index(['Economy','Year'])
            df_list.append(df2)

    # combine individual economy dataframes to one dataframe
    dfResults = pd.concat(df_list, sort=True)
    return dfResults
#    return df2

In [ ]:
# create plotting function

def plot2(economies, df, figurename, Plotylabel, share_x, share_y):
    """
    Line plot for 21 economies. 
    Economies = economies to plot
    df = dataframe of data to plot. Note: each line must be a column.
    Plotylabel = y label for graph
    share_x = share the x axis (true or False)
    share_y = share the y axis (True or False)
    """
    print('Preparing plots...')
    # Create the 'figure'
    plt.style.use('tableau-colorblind10')
    
    # multiple line plot
    fig, axes = plt.subplots(nrows=3, ncols=7, sharex=share_x, sharey=share_y, figsize=(16,12))
    for ax, economy,num in zip(axes.flatten(), economies, range(1,22)):
        print('Creating plot for %s...' %economy)
        df11=df[df['Economy']==economy]
    
        for column in df11.drop(['Economy','Year'], axis=1):
            ax.plot(df11['Year'], df11[column], marker='', linewidth=1.5, label=economy)
            ax.set_title(economy)
            ax.set_ylabel(Plotylabel)
        ax.label_outer()
    
    #plt.tight_layout()
    fig.legend( list(df.drop(['Economy','Year'], axis=1)), bbox_to_anchor=(0,0,1,0.25), loc='lower center', ncol=9)
    fig.savefig(figurename,dpi=200)
    print('Figure saved as %s' % figurename)
    print('Preparing to show the figure...')
    plt.show()

In [ ]:
df = pd.read_csv("../data/raw/Residential/AUS_data.csv",
                index_col=['Economy','Year'])

In [ ]:
df.head()

In [ ]:
economies = df.index.unique(level=0)
economies

In [ ]:
# get list of economies and create economy-model pairs

models = {economy: LinearRegression() for economy in economies}

In [ ]:
x = ['GDP','POP']
y = ['RETT']
df.head()

In [ ]:
run_regression(models,economies,df,x,y)

In [ ]:
# make predictions

df_future = pd.read_csv("../data/raw/Residential/MacroAssumptions_datetime-AUS.csv",
                        index_col=['Economy'])
df_future.head()

In [ ]:
df2 = df_future.drop(columns=["Year"],axis=1)
df2 = df2.rename(columns={"Year2":"Year","Population":"POP"})
df2 = df2.reset_index(drop=False).set_index(['Economy','Year'])

#df2 = df2.drop(columns=["Year"],axis=1)
ResultsColumn = ["y_prediction"]
df2.head()

In [ ]:
results = run_prediction(models, economies, df2, ResultsColumn)
results = results.rename(columns={"Results":"Predicted RETT"})

In [ ]:
results.tail()

In [ ]:
x = results.loc['01_AUS']
x.head()

In [ ]:
fig, ax = plt.subplots()
ax.plot(x.index,x[['Predicted RETT']], 'o', label="Predictions")
#ax.plot(simple_df[['ds']],simple_df[["y"]], 'o', label="Actual")
#ax.legend(loc="best")

# Cleaned up using own functions

In [ ]:
%matplotlib inline

In [ ]:
# import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
def run_regression(models, economies, df, x, y):
    """
    Perform linear regression for one or multiple economies.
    economy = list of economies
    models = {economy: LinearRegression() for economy in economies}
    The function returns a dictionary of economy-model pairs. That is,
    each economy will have its own set of coefficients.
    """
    for economy, model in models.items():
            (model.fit(df.loc[economy, x],
                df.loc[economy, y]))
    return models  

In [ ]:
def run_prediction(models, economies, df, ResultsColumn):
    """
    Use coefficients from run_regression to generate predictions.
    Pass a dataframe df with the X and Y data. 
    ResultsColumn = name of prediction results
    """
    df_list =[]
    # run predictions
    for economy, model in models.items():
            # make prediction
            prediction = model.predict(df.loc[economy,:])
            
            # adding to the input df
            df2 = df.loc[economy,:]
            df2.insert(loc=0,column="Results",value=prediction)
            df2.insert(loc=0,column='Economy',value=economy)
            df2 = df2.reset_index()
            df2 = df2.set_index(['Economy','Year'])
            df_list.append(df2)

    # combine individual economy dataframes to one dataframe
    dfResults = pd.concat(df_list, sort=True)
    return dfResults

In [ ]:
df = pd.read_csv("../data/raw/Residential/AUS_Data.csv",
                index_col=['Economy','Year'])

In [ ]:
economies = df.index.unique(level=0)
economies

In [ ]:
# get list of economies and create economy-model pairs

models = {economy: LinearRegression() for economy in economies}

In [ ]:
x = ['GDP','POP']
y = ['RETT']
df.tail()

In [ ]:
run_regression(models,economies,df,x,y)

In [ ]:
# make predictions

df_future = pd.read_csv("../data/raw/Residential/MacroAssumptions_datetime.csv",
                        index_col=['Economy'])
df_future.tail()

In [ ]:
df2 = df_future.drop(columns=["Year"],axis=1)
df2 = df2.rename(columns={"Year2":"Year","Population":"POP"})
df2 = df2.reset_index(drop=False).set_index(['Economy','Year'])

#df2 = df2.drop(columns=["Year"],axis=1)
ResultsColumn = ["y_prediction"]
df2.tail()

In [ ]:
results = run_prediction(models, economies, df2, ResultsColumn)
results = results.rename(columns={"Results":"Predicted RETT"})
results.tail()

In [ ]:
x = results.loc['01_AUS']
x.head()

In [ ]:
fig, ax = plt.subplots()
ax.plot(x.index,x[['Predicted RETT']], 'o', label="Predictions")